### GMP API

* `__gmpz_init`
* `__gmpz_set_si`
* `__gmpz_add`
* `__gmpz_mul`
* `__gmpz_get_str`

```
typedef struct
{
  int _mp_alloc;                /* Number of *limbs* allocated and pointed
                                   to by the _mp_d field.  */
  int _mp_size;                 /* abs(_mp_size) is the number of limbs the
                                   last field points to.  If _mp_size is
                                   negative this is a negative number.  */
  mp_limb_t *_mp_d;             /* Pointer to the limbs.  */
} __mpz_struct;

typedef mp_limb_t *             mp_ptr;

typedef __mpz_struct *mpz_ptr;

#define mpz_init __gmpz_init
__GMP_DECLSPEC void mpz_init (mpz_ptr);

```


In [1]:
type LargeInt <: Integer
    alloc::Cint
    size::Cint
    d::Ptr{Csize_t}
    
    function LargeInt()
        b = new(zero(Cint), zero(Cint), C_NULL)
        ccall((:__gmpz_init,:libgmp), Void, (Ptr{LargeInt},), &b)
        return b
    end
end
#finalizer(b, _gmp_clear_func)

In [2]:
a=LargeInt()

LargeInt(1,0,Ptr{Uint64} @0x00007fdffff7ca80)

In [3]:
function LargeInt(x::Clong)
    z = LargeInt()
    ccall((:__gmpz_set_si, :libgmp), Void, (Ptr{LargeInt}, Clong), &z, x)
    return z
end

LargeInt (constructor with 2 methods)

In [4]:
LargeInt(10)

LargeInt(1,1,Ptr{Uint64} @0x00007fdfdbd35170)

In [5]:
function +(x::LargeInt, y::LargeInt)
    z=LargeInt()
    ccall((:__gmpz_add, :libgmp), Void, (Ptr{LargeInt}, Ptr{LargeInt}, Ptr{LargeInt}), &z, &x, &y)
    return z
end

+ (generic function with 118 methods)

In [6]:
r=LargeInt(10)+LargeInt(5)

LargeInt(2,1,Ptr{Uint64} @0x00007fdfdbd26fb0)

In [7]:
p = ccall((:__gmpz_get_str,:libgmp), Ptr{Uint8}, (Ptr{Uint8}, Cint, Ptr{LargeInt}), C_NULL, 10, &r)

Ptr{Uint8} @0x00007fdffbf75540

In [8]:
bytestring(p)

"15"

In [9]:
string(x::LargeInt) = bytestring(
           ccall((:__gmpz_get_str,:libgmp), Ptr{Uint8}, (Ptr{Uint8}, Cint, Ptr{LargeInt}), 
                        C_NULL, 10, &x))

string (generic function with 1 method)

In [10]:
string(r)

"15"

In [11]:
Base.show(io::IO, x::LargeInt) = print(io, string(x))

show (generic function with 91 methods)

In [12]:
r

15

In [13]:
LargeInt(10)+LargeInt(5)

15

In [14]:
typemax(Int64) 

9223372036854775807

In [15]:
typemax(Int64) + 1

-9223372036854775808

In [16]:
LargeInt(typemax(Int64))+LargeInt(1)

9223372036854775808

In [17]:
function *(x::LargeInt, y::LargeInt)
    z=LargeInt()
    ccall((:__gmpz_mul, :libgmp), Void, (Ptr{LargeInt}, Ptr{LargeInt}, Ptr{LargeInt}), &z, &x, &y)
    return z
end

* (generic function with 116 methods)

In [18]:
LargeInt(10)*LargeInt(5)

50

In [19]:
Base.convert(::Type{LargeInt}, x::Int64) = LargeInt(x)
Base.promote_rule(::Type{LargeInt}, ::Type{Int64}) = LargeInt

promote_rule (generic function with 114 methods)

In [20]:
LargeInt(10) + 1

11

In [21]:
LargeInt(20) * 20

400

In [24]:
LargeInt(9223372036854775808)

LoadError: `LargeInt` has no method matching LargeInt(::Int128)
while loading In[24], in expression starting on line 1

In [22]:
function LargeInt(s::ASCIIString)
    z = BigInt()
    err = ccall((:__gmpz_set_str, :libgmp),
        Cint, (Ptr{BigInt}, Ptr{Cchar}, Cint),
            &z, s, 10)
    err == 0 || error("invalid big integer: $s")
    return z
end

LargeInt (constructor with 3 methods)

In [23]:
LargeInt("345")

345

In [25]:
LargeInt("9223372036854775808")

9223372036854775808

In [26]:
LargeInt("9223372036854775808") - typemax(Int64) == 1

true